# Main file to run experiments

If you do not know, where to put your code, put it here

In [11]:
from dlpf.base_utils import *

In [12]:
logger = init_log(out_file = 'testbed.log', stderr = False)

In [13]:
import gym
from dlpf.agents import DqnAgent, RandomAgent

## Test with Random agent

Pipeline works, but agent does not learn))

In [14]:
# env = gym.make('PathFindingByPixel-v1')
# env.configure(tasks_dir = os.path.abspath('data/sample/imported/'), scale = 10, map_shape = (10, 10))
# env.monitor.start('data/sample/results/random', force=True, seed=0)

# agent = RandomAgent(env.action_space)

# episode_count = 1
# max_steps = 10
# reward = 0
# done = False

# for i in range(episode_count):
#     ob = env.reset()

#     for j in range(max_steps):
#         action = agent.act(ob, reward, done)
#         ob, reward, done, _ = env.step(action)
#         if done:
#             break
#         # Note there's no env.render() here. But the environment still can open window and
#         # render if asked by env.monitor: it calls env.render('rgb_array') to record video.
#         # Video is not recorded every episode, see capped_cubic_video_schedule for details.

# # Dump result info to disk
# env.monitor.close()

## Basic DQN

https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [15]:
env = gym.make('PathFindingByPixel-v1')
env.configure(tasks_dir = os.path.abspath('data/sample/imported/'), monitor_scale = 10, map_shape = (10, 10))

In [16]:
env.monitor.start('data/sample/results/basic_dqn', force=True, seed=0)

In [17]:
agent = DqnAgent(state_size = env.observation_space.shape,
                 number_of_actions = env.action_space.n,
                 save_name = env.__class__.__name__)

Training a new model


In [18]:
episode_count = 5000
max_steps = 100

for _ in xrange(episode_count):
    observation = env.reset()
    agent.new_episode()
    for _ in range(max_steps):
        action, values = agent.act(observation)
        observation, reward, done, info = env.step(action)
        agent.observe(reward)
        if done:
            break

In [19]:
!rm PathFindingByPixelEnv.h5